In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from PIL import Image, ImageDraw, ImageFont
import json
import random
import gradio as gr
import queue

import clip
import os
from tqdm import tqdm
from fine_tune import draw_text_with_new_lines, MyDataset, TestDataset, calculate_corr, load_model, evaluate, all_attributes, exclusive_attributes, inclusive_attributes
from cj_fonts import inclusive_fonts, fifty_fonts
from IPython.display import display
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as font_manager

# If using GPU then use mixed precision training.
device = "cuda:0" if torch.cuda.is_available() else "cpu"
# Must set jit=False for training
model, preprocess = clip.load("ViT-B/32", device=device, jit=False)

from torchvision.transforms.functional import pil_to_tensor, to_pil_image

In [2]:
char_size = 150
font_dir = '../gwfonts'
cj_font_dir = '../all-fonts'
font_paths = [os.path.join(font_dir, f) for f in os.listdir(font_dir)]
cj_font_paths = [os.path.join(cj_font_dir, f) for f in os.listdir(cj_font_dir)]

# add font
for font in font_manager.findSystemFonts(font_dir):
    font_manager.fontManager.addfont(font)

for font in font_manager.findSystemFonts(cj_font_dir):
    font_manager.fontManager.addfont(font)

ttf_list = font_manager.fontManager.ttflist

predicted_attributes = json.load(open('../attributeData/predicted_cj_font_attribute.json', 'r'))
attribute_to_indexes = {attribute: all_attributes.index(attribute) for attribute in all_attributes}

In [3]:
"""
def draw_text_with_new_lines(text, font, img_width, img_height):
    # (left, top, right, down)
    _, top, _, _ = font.getbbox(text)
    print(top)
    image = Image.new('RGB', (img_width, img_height), color=(255, 255, 255))
    draw = ImageDraw.Draw(image)
    lines = text.split('\n')
    y_text = 0
    for line in lines:
        line_width, line_height = font.getsize(line)
        draw.text(((img_width - line_width) / 2, y_text-top+15),
                  line, font=font, fill=(0, 0, 0))
        y_text += line_height
    return image
"""

"\ndef draw_text_with_new_lines(text, font, img_width, img_height):\n    # (left, top, right, down)\n    _, top, _, _ = font.getbbox(text)\n    print(top)\n    image = Image.new('RGB', (img_width, img_height), color=(255, 255, 255))\n    draw = ImageDraw.Draw(image)\n    lines = text.split('\n')\n    y_text = 0\n    for line in lines:\n        line_width, line_height = font.getsize(line)\n        draw.text(((img_width - line_width) / 2, y_text-top+15),\n                  line, font=font, fill=(0, 0, 0))\n        y_text += line_height\n    return image\n"

In [4]:
def choose_random_font_pairs(font_dir):
    font_paths = [os.path.join(font_dir, f) for f in os.listdir(font_dir)]
    font_path1 = random.choice(font_paths)
    font_path2 = random.choice(font_paths)
    while font_path1 == font_path2:
        font2_path2 = random.choice(font_paths)
    return font_path1, font_path2

In [5]:
def choose_font_pairs_in_order(font_dir):
    font_paths = [os.path.join(font_dir, f) for f in os.listdir(font_dir)]
    for font_path1 in font_paths:
        for font_path2 in font_paths:
            if font_path1 != font_path2:
                yield font_path1, font_path2

In [6]:
def choose_font_pairs_same_apper_times_with_queue(font_paths, appear_times=8, target_font_num=50, random_seed = 123):
    sorted_font_paths = sorted(font_paths)

    random.seed(random_seed)
    font_paths = sorted_font_paths[:target_font_num] * appear_times
    random.shuffle(font_paths)

    font_paths_queue = queue.Queue()
    for font_path in font_paths:
        font_paths_queue.put(font_path)
    flag = False
    count = 0
    # extract two font paths from font_paths_queue
    while True:
        if font_paths_queue.qsize() < 2:
            break
        if count > 1000:
            break

        font_path2 = font_paths_queue.get()
        if not flag:
            font_path1 = font_paths_queue.get()
        flag = False
        if font_path1 != font_path2:
            print(font_path1, font_path2)
            yield font_path1, font_path2
        else:
            count += 1
            font_paths_queue.put(font_path2)
            flag = True

In [7]:
def choose_font_pairs_same_apper_times(font_dir, appear_times=8, target_font_num=50, random_seed=123):
    sorted_font_paths = sorted([os.path.join(font_dir, f) for f in os.listdir(font_dir)])

    random.seed(random_seed)
    random.shuffle(sorted_font_paths)
    font_paths = sorted_font_paths[:target_font_num] * appear_times

    font_paths_queue = queue.Queue()
    for font_path in font_paths:
        font_paths_queue.put(font_path)
    flag = False

    # extract two font paths from font_paths_queue
    i = 0
    while True:
        if i > len(font_paths) - 1:
            break
        font_path1 = font_paths[i]
        font_path2 = font_paths[i+1]
        i += 2
        if font_path1 != font_path2:
            yield font_path1, font_path2

In [8]:
def choose_random_target_attribute(target_attributes):
    target_attribute = random.choice(target_attributes)
    return target_attribute

In [9]:
text = '春'
text = '山路を登りながら\nこう考えた\n智に働けば角が立つ'
text = 'あのイーハトーヴォの\nすきとおった風、\n夏でも底に冷たさをもつ\n青いそら'
text = '春夏秋冬\n世界平和'
text = '春夏\n秋冬'
top5_attributes = ['thin', 'calm', 'sloppy', 'complex', 'strong',]
bottom5_attributes = ['happy', 'fresh', 'gentle', 'sharp', 'technical',]
#target_attributes = top5_attributes
target_attributes = ['thin', 'calm', 'sloppy', 'sharp', 'technical', 'traditional', 'Japanese_style', 'robust']
target_attribute = target_attributes[0]
fifty_cj_font_paths = [os.path.join(cj_font_dir, f) for f in fifty_fonts.split('\n') if f != '']
target_font_num = 50
#target_font_num = 10
appear_times = 2
share = True


line_num = text.count('\n') + 1
default_width = int((char_size+50) * len(text) / line_num)
default_height = int((char_size+50) * line_num)
#font_paris_generator = choose_font_pairs_in_order(cj_font_dir)

# generate random seed by converting target_attribute to int
random_seed = int.from_bytes(target_attribute.encode(), 'little')

font_paris_generator = choose_font_pairs_same_apper_times_with_queue(fifty_cj_font_paths, appear_times=appear_times, target_font_num=target_font_num, random_seed = random_seed)
total_pair_num = sum([1 for _ in font_paris_generator])
print('total_pair_num', total_pair_num)
font_paris_generator = choose_font_pairs_same_apper_times_with_queue(fifty_cj_font_paths, appear_times=appear_times, target_font_num=target_font_num, random_seed = random_seed)

default_font_path1, default_font_path2 = next(font_paris_generator)
print(default_font_path1)
default_font1 = ImageFont.truetype(default_font_path1, char_size)
default_font2 = ImageFont.truetype(default_font_path2, char_size)
navigation_font_path = os.path.join(cj_font_dir, 'ipaexg.ttf')
navigation_font = ImageFont.truetype(navigation_font_path, char_size)
navigation_width = char_size * 15
navigation_height = char_size * 2

global_font_name1 = os.path.splitext(os.path.basename(default_font_path1))[0]
global_font_name2 = os.path.splitext(os.path.basename(default_font_path2))[0]

annotation_result = []
target_attribute_count = 0
current_num = 0
save_flags = []

def create_attribute_markdown(target_attribute):
    #return f'<div style="font-size: 50px; text-align: center">Which font is more {target_attribute}?</div>'
#     return f'Which is more {target_attribute}?'
    return f'{target_attribute}?'

def draw_percentage_bar(current_num, total_num=total_pair_num, size=(300, 10), background_color=(255, 255, 255), bar_color=(186, 251, 176), total_bar_color=(131, 237, 231)):
    global target_attribute_count

    if current_num == 0 and target_attribute_count == 0:
        return Image.new('RGB', size, background_color)
    elif current_num == 0:
        percentage = (target_attribute_count + 1) / len(target_attributes)
        bar_width = int(size[0] * percentage)
        bar_height = int(size[1] / 3)

        image = Image.new('RGB', size, background_color)
        draw = ImageDraw.Draw(image)
        draw.rectangle((0, 0, bar_width, bar_height), fill=total_bar_color)

    percentage = current_num / total_num
    image = Image.new('RGB', size, background_color)
    draw = ImageDraw.Draw(image)
    bar_width = int(size[0] * percentage)
    bar_height = int(size[1] / 3)
    draw.rectangle((0, bar_height, bar_width, int(size[1])), fill=bar_color)
    if target_attribute_count == 0:
        total_bar_width = 0
    else:
        total_percentage = (target_attribute_count) / len(target_attributes)
        total_bar_width = int(size[0] * total_percentage)
        draw.rectangle((0, 0, total_bar_width, bar_height), fill=total_bar_color)

    return image


def builder(result, name, text=text, char_size=char_size):
    global global_font_name1, global_font_name2, current_num, target_attribute_count, annotation_result, font_paris_generator

    # set up target_attribute
    target_attribute = target_attributes[target_attribute_count]

    # set up percentage bar
    current_num += 1
    percentage_image = draw_percentage_bar(current_num)

    # set annotation_result
    annotation_result.append((global_font_name1, global_font_name2, target_attribute, result))

    #font_path1, font_path2 = choose_random_font_pairs(font_dir)
    try:
        font_path1, font_path2 = next(font_paris_generator)
    except StopIteration:
        print('StopIteration')

        # save annotation_result
        if target_attribute not in save_flags:
            rator_name = name.replace(' ', '_')
            output_dir = '../attributeData/outputs'
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)
            output_path = os.path.join(output_dir, f'{rator_name}-{target_attribute}-{target_font_num}.json')
            with open(output_path, 'w') as f:
                json.dump(annotation_result, f)
            save_flags.append(target_attribute)
            annotation_result = []

        # update target_attribute
        if target_attribute_count == len(target_attributes) - 1:
            # trick
            target_attribute_count += 1
            percentage_image = draw_percentage_bar(current_num)
            target_attribute_count -= 1

            image1 = draw_text_with_new_lines('Finish!', navigation_font, navigation_width, navigation_height)
            image2 = draw_text_with_new_lines('Thank you!', navigation_font, navigation_width, navigation_height)
            return image1, image2, percentage_image, create_attribute_markdown(target_attribute)
        else:
            target_attribute_count += 1
            image1 = draw_text_with_new_lines(f'Next is {target_attributes[target_attribute_count]}', navigation_font, navigation_width, navigation_height)
            image2 = draw_text_with_new_lines('Push Next', navigation_font, navigation_width, navigation_height)
            current_num = 0

            #random_seed = random.randint(0, 1000)
            random_seed = int.from_bytes(target_attribute.encode(), 'little')
            font_paris_generator= choose_font_pairs_same_apper_times_with_queue(fifty_cj_font_paths, appear_times=appear_times, target_font_num=target_font_num, random_seed=random_seed)
            return image1, image2, percentage_image, create_attribute_markdown(target_attribute)

    font1 = ImageFont.truetype(font_path1, char_size)
    font2 = ImageFont.truetype(font_path2, char_size)
    font_name1 = os.path.splitext(os.path.basename(font_path1))[0]
    font_name2 = os.path.splitext(os.path.basename(font_path2))[0]
    global_font_name1 = font_name1
    global_font_name2 = font_name2


    line_num = text.count('\n') + 1
    width = int((char_size+50) * len(text) / line_num)
    height = int((char_size+50) * line_num)

    image1 = draw_text_with_new_lines(text, font1, width, height)
    image2 = draw_text_with_new_lines(text, font2, width, height)
    return image1, image2, percentage_image, create_attribute_markdown(target_attribute)

with gr.Blocks() as demo:

    with gr.Row():
        name = gr.Text(value='Your Name', label='Name', interactive=True)
    with gr.Row():
        percentage_image = gr.Image(value=draw_percentage_bar(0), label='Progress', interactive=False,)
    with gr.Row():
        image1 = gr.Image(value=draw_text_with_new_lines(text, default_font1, default_width, default_height), label='Font A', interactive=False)
        image2 = gr.Image(value=draw_text_with_new_lines(text, default_font2, default_width, default_height), label='Font B', interactive=False)

    with gr.Row():
        #target_attribute_text = gr.Text(value=target_attribute, label='Target Attribute', interactive=False, visible=False)
        #attribute_markdown = gr.Markdown(value=f'<div style="font-size: 50px; text-align: center">Which font is more {target_attribute}?</div>', label='Target Attribute', interactive=False)
        attribute_markdown = gr.Text(value=f'{target_attribute}?', label='Which is more', interactive=False)
        #target_attribute = gr.Text(value=choose_random_target_attribute(target_attributes), label='Target Attribute', interactive=False)

    with gr.Row():
        #check = gr.Radio(choices=['Font A', 'slightly, Font A', 'Equal', 'slightly, Font B', 'Font B'], value = 'Eaqual', label=f'', interactive=True)
        check = gr.Radio(choices=['Font A', 'Font B'], value = 'Font A', label=f'', interactive=True)

    with gr.Row():
        next_button  = gr.Button(value='Next', interactive=True)
        #next_button.click(builder, inputs=[check, target_attribute], outputs=[target_attribute, image1, image2])
    next_button.click(builder, inputs=[check, name], outputs=[image1, image2, percentage_image, attribute_markdown], show_progress=False)



demo.launch(debug=True, share=share)

../all-fonts/JP_Ronde-B_square.otf ../all-fonts/SanariFontB001.ttf
../all-fonts/ZenOldMincho-Black.ttf ../all-fonts/HanyiSentyPastel.ttf
../all-fonts/crayon_1-1.ttf ../all-fonts/SentyCaramel.ttf
../all-fonts/kiloji_d.ttf ../all-fonts/irohamaru-Medium.ttf
../all-fonts/Tanugo-TTF-Regular.ttf ../all-fonts/YuseiMagic-Regular.ttf
../all-fonts/chogokubosogothic_5.ttf ../all-fonts/g_pencilkaisho_free.ttf
../all-fonts/HanyiSentyPastel.ttf ../all-fonts/ゆず ポップ A [M] Light.ttf
../all-fonts/HachiMaruPop-Regular.ttf ../all-fonts/crayon_1-1.ttf
../all-fonts/KFhimaji.otf ../all-fonts/Corporate-Logo-Medium-ver3.otf
../all-fonts/kiloji_d.ttf ../all-fonts/Kazesawa-ExtraLight.ttf
../all-fonts/Corporate-Logo-Medium-ver3.otf ../all-fonts/chinese-handwriting-style.ttf
../all-fonts/SentyGoldSand.ttf ../all-fonts/Chalk-S-JP.otf
../all-fonts/g_comickoin_freeL.ttf ../all-fonts/35.ttf
../all-fonts/g_comickoin_freeB.ttf ../all-fonts/ZCOOLXiaoWei-Regular.ttf
../all-fonts/KodomoRounded.otf ../all-fonts/M+A1_heav

../all-fonts/ZenOldMincho-Black.ttf ../all-fonts/HanyiSentyPastel.ttf
../all-fonts/crayon_1-1.ttf ../all-fonts/SentyCaramel.ttf
../all-fonts/kiloji_d.ttf ../all-fonts/irohamaru-Medium.ttf
../all-fonts/Tanugo-TTF-Regular.ttf ../all-fonts/YuseiMagic-Regular.ttf
../all-fonts/chogokubosogothic_5.ttf ../all-fonts/g_pencilkaisho_free.ttf
../all-fonts/HanyiSentyPastel.ttf ../all-fonts/ゆず ポップ A [M] Light.ttf
../all-fonts/HachiMaruPop-Regular.ttf ../all-fonts/crayon_1-1.ttf
../all-fonts/KFhimaji.otf ../all-fonts/Corporate-Logo-Medium-ver3.otf
../all-fonts/kiloji_d.ttf ../all-fonts/Kazesawa-ExtraLight.ttf
../all-fonts/Corporate-Logo-Medium-ver3.otf ../all-fonts/chinese-handwriting-style.ttf
../all-fonts/SentyGoldSand.ttf ../all-fonts/Chalk-S-JP.otf
../all-fonts/g_comickoin_freeL.ttf ../all-fonts/35.ttf
../all-fonts/g_comickoin_freeB.ttf ../all-fonts/ZCOOLXiaoWei-Regular.ttf
../all-fonts/KodomoRounded.otf ../all-fonts/M+A1_heavy-10-1.2.otf
../all-fonts/NasuM-Bold-20200227.ttf ../all-fonts/Sent

In [ ]:
name='Takahori_Hosokawa'
target_attribute = 'thin'
with open(f'../attributeData/outputs/{name}-{target_attribute}-{target_font_num}.json', 'r') as f:
    results = json.load(f)

FileNotFoundError: [Errno 2] No such file or directory: '../attributeData/outputs/Takahori_Hosokawa-thin-50.json'

In [45]:
len(results)

50